In [237]:
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd

* ### Аналитическое решение

In [238]:
def analytic_solution(x,t):
    return math.cos(x+2*t)

In [239]:
def analytic_points(t):
        x1 = np.linspace(0,1,11)
        u = []
        for x in x1:
                res = analytic_solution(x,t)
                u.append(res)
        return x1,u

* ### Численное решение

In [346]:
def solve(L,T):
    tau = 1/T
    h = 1/L
    solutions = [] 
    x = np.linspace(0,1,L+1)
    t = np.linspace(0,1,T+1)
    u_next = lambda curr,next1,next2,next3,tau,h: curr + tau/(3*h)*(2*next3-9*next2+18*next1-11*curr)+2*(tau**2)/h**2 *(-next3+4*next2-5*next1+2*curr) + 4*tau**2/(3*h**2) * (next3-3*next2+3*next1-curr)
    u_l0 = lambda x: math.cos(x)
    u_Ln = lambda t: math.cos(1+2*t)
    u = np.zeros([len(x),len(t)]); ux0 = []; uLn = []
    for i in range(0,len(x)): 
            u[i][0] = u_l0(x[i])
    for j in range(1,len(t)):
            u[len(x)-1][j] = u_Ln(t[j])
    for j in range(0,len(t)-1):
        u[len(x)-2][j+1] = math.cos(1+2*t[j+1]) + h*math.sin(1+2*t[j+1])-h**2/2*math.cos(1+2*t[j+1])-(h**3)/6 * math.sin(1+2*t[j+1])
        u[len(x)-3][j+1] = math.cos(1+2*t[j+1]) + 2*h*math.sin(1+2*t[j+1])-2*(h**2)*math.cos(1+2*t[j+1])-4/3*(h**3) * math.sin(1+2*t[j+1])
        for i in range(0,len(x)-3):
            u[i][j+1] = u_next(u[i][j],u[i+1][j],u[i+2][j],u[i+3][j],tau,h)
    
    setka = np.linspace(0,1,11)
    setka = np.round(setka,15)
    setka = list(setka)
    x = np.round(x,15)
    x = list(x)
    indexes = [x.index(i) for i in setka]
    solutions = [u[ind][len(t)-1] for ind in indexes]
     
    return solutions

* ### Таблица результатов

In [350]:
x,u1 = analytic_points(1)
L = 1000; T = 1000
u2 = solve(L,T)
delta1 = []; 
for i in range(len(u1)):
    delta1.append(abs(u1[i] - u2[i]))
pd.options.display.float_format = "{:.4e}".format
data = pd.DataFrame({"x": x, "u аналит.": u1, "u числ. ": u2 ,"u числ.- u аналит.": delta1})
display(data)
print("Максимальная норма ошибки =", max(delta1))

,x,u аналит.,u числ.,u числ.- u аналит.
0,0.0000e+00,-4.1615e-01,-4.1615e-01,2.8227e-13
1,1.0000e-01,-5.0485e-01,-5.0485e-01,3.2585e-13
2,2.0000e-01,-5.8850e-01,-5.8850e-01,3.7115e-13
3,3.0000e-01,-6.6628e-01,-6.6628e-01,4.3088e-13
4,4.0000e-01,-7.3739e-01,-7.3739e-01,4.9283e-13
5,5.0000e-01,-8.0114e-01,-8.0114e-01,5.2203e-13
6,6.0000e-01,-8.5689e-01,-8.5689e-01,5.6466e-13
7,7.0000e-01,-9.0407e-01,-9.0407e-01,6.0696e-13
8,8.0000e-01,-9.4222e-01,-9.4222e-01,6.3127e-13
9,9.0000e-01,-9.7096e-01,-9.7096e-01,6.4093e-13


Максимальная норма ошибки = 6.409317521161029e-13


* ### Исследование на устойчивость

In [242]:
def courant_num(L,T):
    return L/T

In [251]:
def check(deltas,pairs):   ### вычисление числа Куранта при котором ошибка начинает возрастать
    for i in range(len(deltas)-1):
        if deltas[i+1] > deltas[i]:
            return courant_num(pairs[i+1][0],pairs[i+1][1]) 

In [351]:
L = 10**2; T1 = 10**5; T2 = 10**4; pairs1 = [[L,T1]]; pairs2 = [[L,T2]]
while(courant_num(L,T1) <= 1):
    T1 = T1/2; pairs1.append([L,T1])
while(courant_num(L,T2) >= 0.001):
    T2 = T2*2; pairs2.append([L,T2])

In [352]:
x,u11 = analytic_points(1); deltas1 = []; deltas2 = []; delta1 = []; delta2 = []; courants1 = []; courants2 = []

In [353]:
for pair in pairs1:
    courants1.append(courant_num(pair[0],pair[1]))
    u21 = solve(int(pair[0]),int(pair[1]))
    for i in range(0,len(u11)):
        delta1.append(abs(u11[i] - u21[i]))
    deltas1.append(max(delta1))

In [354]:
df = pd.DataFrame({"Число Куранта": courants1, "Максимальная ошибка": deltas1})
display(df)
print("Решение неустойчиво при числе Куранта = ", check(deltas1,pairs1))

,Число Куранта,Максимальная ошибка
0,1.0000e-03,7.2509e-08
1,2.0000e-03,7.2509e-08
2,4.0000e-03,1.3638e-07
3,8.0000e-03,3.6989e-07
4,1.6000e-02,8.3133e-07
5,3.2000e-02,1.7319e-06
6,6.4000e-02,3.4450e-06
7,1.2800e-01,6.5124e-06
8,2.5600e-01,1.3124e-05
9,5.1200e-01,3.9243e+14


Решение неустойчиво при числе Куранта =  0.004
